In [16]:
!pip install tavily-python

In [39]:
%set_env OPENAI_API_KEY=
TAVILY_API_KEY="tvly-"

env: OPENAI_API_KEY=


In [4]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage

memory = SqliteSaver.from_conn_string(":memory:")

In [37]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [5]:
class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    critique: str
    content: List[str]
    revision_number: int
    max_revisions: int

In [6]:
PLAN_PROMPT = """You are an expert writer tasked with writing a high level outline of an essay. \
Write such an outline for the user provided topic. Give an outline of the essay along with any relevant notes \
or instructions for the sections."""

In [7]:
WRITER_PROMPT = """You are an essay assistant tasked with writing excellent 2-paragraph essays.\
Generate the best essay possible for the user's request and the initial outline. \
If the user provides critique, respond with a revised version of your previous attempts. \
Utilize all the information below as needed: 

------

{content}"""

In [8]:
REFLECTION_PROMPT = """You are a teacher grading an essay submission. \
Generate critique and recommendations for the user's submission. \
Provide detailed recommendations, including requests for length, depth, style, etc."""

In [9]:
RESEARCH_PLAN_PROMPT = """You are a researcher charged with providing information that can \
be used when writing the following essay. Generate a list of search queries that will gather \
any relevant information. Only generate 3 queries max."""


In [10]:
RESEARCH_CRITIQUE_PROMPT = """You are a researcher charged with providing information that can \
be used when making any requested revisions (as outlined below). \
Generate a list of search queries that will gather any relevant information. Only generate 3 queries max."""

In [11]:
from langchain_core.pydantic_v1 import BaseModel

class Queries(BaseModel):
    queries: List[str]

In [20]:
from tavily import TavilyClient
import os
tavily = TavilyClient(api_key=TAVILY_API_KEY)

In [21]:
def plan_node(state:AgentState):
    messages = [
        SystemMessage(content=PLAN_PROMPT),
        HumanMessage(content=state['task'])
    ]
    response =  model.invoke(messages)
    return {'plan':response.content}

In [22]:
def research_plan_node(state:AgentState):
    messages= [
        SystemMessage(content=RESEARCH_PLAN_PROMPT),
        HumanMessage(content=state['task'])
    ]
    queries = model.with_structured_output(Queries).invoke(messages)
    content =  state['content'] or []
    for q in queries.queries:
        response = tavily.search(query=q)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [23]:
def generation_node(state:AgentState):
    content = '\n\n'.join(state['content']) or []
    user_message = HumanMessage(
        content=f"{state['task']}\n\nHere is my plan:\n\n{state['plan']}")
    messages = [
        SystemMessage(
            content=WRITER_PROMPT.format(content=content)
        ),
        user_message
        ]
    response = model.invoke(messages)
    return {
        "draft": response.content, 
        "revision_number": state.get("revision_number", 1) + 1
    }

In [24]:
def reflection_node(state:AgentState):
    messages = [
        SystemMessage(content=REFLECTION_PROMPT), 
        HumanMessage(content=state['draft'])
    ]
    response = model.invoke(messages)
    return {"critique": response.content}

In [25]:
def research_critique_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_CRITIQUE_PROMPT),
        HumanMessage(content=state['critique'])
    ])
    content = state['content'] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [26]:
def should_continue(state):
    if state["revision_number"] > state["max_revisions"]:
        return END
    return "reflect"

In [27]:
builder = StateGraph(AgentState)

In [28]:
builder.add_node("planner",plan_node)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_node("research_plan", research_plan_node)
builder.add_node("research_critique", research_critique_node)

In [29]:
builder.set_entry_point("planner")

In [30]:
builder.add_conditional_edges(
    "generate", 
    should_continue, 
    {END: END, "reflect": "reflect"}
)


In [31]:
builder.add_edge("planner","research_plan")
builder.add_edge("research_plan","generate")

In [32]:
builder.add_edge("reflect", "research_critique")
builder.add_edge("research_critique", "generate")

In [34]:
graph = builder.compile(checkpointer=memory)

In [35]:
thread = {"configurable": {"thread_id": "1"}}

In [38]:
for s in graph.stream({
    'task': "what is the difference between Andhra Pradesh & Telangana",
    "max_revisions": 2,
    "revision_number": 1,
}, thread):
    print(s)

{'planner': {'plan': 'I. Introduction\n    A. Brief overview of the historical background of Andhra Pradesh and Telangana\n    B. Thesis statement: Despite being part of the same region, Andhra Pradesh and Telangana have distinct differences in terms of history, culture, and governance.\n\nII. Historical Background\n    A. Formation of Andhra Pradesh in 1956\n    B. Separation of Telangana from Andhra Pradesh in 2014\n\nIII. Cultural Differences\n    A. Language: Telugu in Andhra Pradesh, Telugu and Urdu in Telangana\n    B. Festivals and traditions unique to each region\n    C. Cuisine variations\n\nIV. Governance and Administration\n    A. Administrative divisions and capitals\n    B. Political landscape and major political parties in each state\n    C. Economic disparities and development initiatives\n\nV. Socio-Economic Factors\n    A. Demographic differences\n    B. Education and healthcare infrastructure\n    C. Employment opportunities and industrial growth\n\nVI. Current Issues